## Opinion Mining

Please enter the topics for which people complaints from forums are required

In [1]:
topics = ['Problems of social media', 'asd', '']

In [2]:
#Import Section
from urllib import request
from bs4 import BeautifulSoup as soup
import pandas as pd
from selenium import webdriver
import requests
import TextAnalyzer
import time
import multiprocessing
import re

In [3]:
# Create Selenium chrome driver
driver = webdriver.Chrome(executable_path = r'C:\Users\nithin.prakash\Desktop\chromedriver_win32/chromedriver.exe')

# Url from which we scrape data
base_url = 'https://www.quora.com/search?q='
NUM_POSTS = 10

In [4]:
# function to get list for questions for a topic
quora_questions = pd.DataFrame(columns=["Topic", "Question", "Link"])
def get_questions(topic, url):
    try:
        # Make a request to URL to get NUM_POSTS
        driver.get(url)
        posts = driver.find_elements_by_css_selector("div.pagedlist_item")

        timeout = time.time() + 200   # 200 seconds from now
        while len(posts) < NUM_POSTS and time.time() < timeout:
            driver.execute_script("arguments[0].scrollIntoView();", posts[-1])

            posts = driver.find_elements_by_css_selector("div.pagedlist_item")
        # Convert selenium to BeautifulSoup object
        page_soup = soup(driver.page_source, 'html.parser')
        page_list_item = page_soup.findAll('div', {'class': 'pagedlist_item'})

        # Get questions and links from HTML     
        for item in page_list_item:
            question = item.find('span', {'class': 'ui_qtext_rendered_qtext'})
            if question:
                link = item.find('a', href=True)
                if link:
                    row = [topic, question.text, 'https://www.quora.com' + link['href']]
                    quora_questions.loc[len(quora_questions)] = row
    except: 
        return

In [5]:
#get questions for all topics in the list
for topic in topics:
    topic_modified = topic.replace(" ", "+")
    url = base_url + topic_modified 
    get_questions(topic, url)
;

    

In [6]:
# Store questions and links to an excel
quora_questions.to_csv('Quora_Questions.csv', encoding='utf-8', index=False)

# Access each questions one at a time and scrape comments
quora_comments = pd.DataFrame(columns=["Topic", "Comment", "Date", "Emotion", "Sentiment", "Link", 'Question'])
for index, data in quora_questions.iterrows():
    try:
        #Call the post and get HTML content
        print(data['Link'])
        response = requests.get(data['Link'].encode('utf-8'))

        #Convert to BeautifulSoup object
        page_soup = soup(response.text, 'html.parser')

        # get Topic details
        topics_list = page_soup.findAll('span', {'class': 'TopicNameSpan TopicName'})
        topics = ""
        if topics_list:
            for topic in topics_list:
                topics += topic.text + ", "

        #Get Comment details and store in a dataframe
        list_item = page_soup.findAll('div', {'class': 'pagedlist_item'}) 
        if list_item:
            for item in list_item:        
                answer_date = item.find("a", {"class": "answer_permalink"})
                if answer_date:            
                    text_expanded = item.findAll("div", {"class": "ui_qtext_expanded"})   
                    comment = ""
                    if text_expanded:
                        comment = text_expanded[0].text
                    comment_date = str(answer_date.text)
                    row = [data['Topic'], comment, comment_date , " ", " ", data['Link'], data['Question']]
                    quora_comments.loc[len(quora_comments)] = row
    except:
        print("error")

https://www.quora.com/Is-Social-Media-addiction-a-real-problem
https://www.quora.com/Why-is-Facebook-considered-social-media
https://www.quora.com/How-can-we-solve-this-problem-of-fake-social-media-profiles
https://www.quora.com/How-does-social-media-affect-mental-health-1
https://www.quora.com/unanswered/How-can-we-harness-the-power-of-social-media-platforms-to-crowdsource-solutions-to-serious-problems-of-society
https://www.quora.com/Besides-other-factors-is-social-media-a-cause-of-anxiety-depression-and-modern-day-psychological-health-problems
https://www.quora.com/Is-posting-problems-on-social-media-with-movements-like-MeToo-really-helpful
https://www.quora.com/What-are-your-top-3-social-media-marketing-challenges
https://www.quora.com/Whats-a-first-world-problem-that-you-face-every-day
https://www.quora.com/What-are-the-biggest-problems-with-current-social-media-sites-Facebook-Twitter-etc
https://www.quora.com/Why-do-people-share-their-problems-on-social-media
https://www.quora.co

In [7]:
#Store all the details in excel
quora_comments = quora_comments.loc[quora_comments['Comment'] != "" ]
quora_comments.to_csv('Quora_comments.csv', encoding='utf-8', index=False)

In [8]:
quora_comments = pd.read_csv('Quora_comments.csv')

In [9]:
#Get emotions and sentiments for each comments

for index, row in quora_comments.iterrows():
    text_analysed=TextAnalyzer.textAnalyzer(row['Comment'])
    row['Emotion'] = text_analysed.emotions()
    row['Sentiment'] = text_analysed.sentiments_new()

C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users

C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users

C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users

C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users

C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users

C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users\nithin.prakash\Desktop\Hackathon\TextAnalyzer.py:107: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  url = 'https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
C:\Users

In [10]:
#Store all the details in excel
quora_comments.to_csv('Quora_comments.csv', encoding='utf-8', index=False)

In [11]:
quora_comments = pd.read_csv('Quora_comments.csv')

Filter processed Data based on on below criteria

- Length of teh comment should be more than 30 characters
- Sentiment of teh comment should be Very Dissatisfied
- COmment should be rlevant to the topic

In [12]:
filtered_comments = quora_comments[(len(quora_comments['Comment']) > 30) & (quora_comments['Sentiment'] == 'Very Dissatisfied')]

In [13]:
filt_topics = []
topic_keys = []
for topic in topics:
    fir_key = topic.replace('Problems', '').replace('issues', '')
    filt_topics.append(fir_key)

for topic in filt_topics:
    key = max(re.findall(r'\S+',topic), key = len)
    topic_keys.append(key)
    sec_key = topic.replace(key, "")
    key = max(re.findall(r'\S+',sec_key), key = len)
    topic_keys.append(key.lower())

ValueError: max() arg is an empty sequence

In [ ]:
def filter_comments_emotion(emotion):
    sadness = emotion['sadness']
    joy = emotion['joy']
    fear = emotion['fear']
    disgust = emotion['disgust']
    anger = emotion['anger']
    if joy > sadness or joy > fear or joy > disgust or joy > anger:
        return True
    return False

def filter_topic_keys(comment):
    comment = comment.lower()
    words = comment.split(" ")
    i = 0;
    for word in words:
        for key in topic_keys:
            if key in word:
                i+=1
    if i > 1:
        return True
    return False

In [ ]:
#Filter based on Relevance
filtered_comments['is_relevant'] = [filter_topic_keys(x) for x in filtered_comments['Question']]
filtered_comments = filtered_comments[(filtered_comments['is_relevant'] == True)]

In [14]:
filtered_comments.to_csv('filtered_comments.csv')

#### Call Web API to create Ticket

In [15]:
# api-endpoint 
URL = "https://evry-hackathaon-ticket.azurewebsites.net/api/Issue"
  
headers = {'Content-type': 'application/json'}
for index,row in filtered_comments.iterrows():
        
    ans_date = row['Date'].replace('Answered', '').replace('Updated', '')


    # defining a params dict for the parameters to be sent to the API 
    data = {'IssueDate':ans_date.strip(), 'IssueLink': row['Link'].strip(),
             "Topic": row['Topic'].strip(),
            "IssueTitle": row['Comment'][:60].strip(),
            "Comment": row['Comment']}

    # sending get request and saving the response as response object 
    r = requests.post(url = URL, json = data, headers = headers) 